In [1]:
from os import listdir
from os.path import isfile, join

def get_all_files_from_path(mypath):
    filenames = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
    return filenames

from bs4 import BeautifulSoup
import re
import json

def get_article(articles):
    result = {}
    current_statue = "(non-statute)"
    for i in re.split(r"(.*)", articles.strip()):
        if len(i) == 0 or i == "\n":
            continue
        if re.search(r"^\(.*\)$", i):
            current_statue = i.strip()
            if current_statue not in result:
                result.update({current_statue: []})
        else:
            if current_statue not in result:
                result.update({current_statue: []})
            result[current_statue].append(i)
    return result

def build_test(filename):
    result = {}
    with open(filename, 'r') as f:
        data = f.read()

    data = BeautifulSoup(data, "xml").find_all('pair')
    for i in data:
        id = i.get('id')
        result.update({id: {}})
        result[id].update({"label": i.get('label')})
        articles = i.find('t1').text.strip()
        # articles = get_article(articles)
        result[id].update({"result": articles})
        result[id].update({"content": i.find('t2').text.strip()})
    return result

def write_json(filename, data):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

import xml.etree.ElementTree as Et
import glob

def format_first_line(text):
    lines = text.split("\n")
    results = []
    for line in lines:
        if line[0] == "":
            continue
        if line[0] == "(" and line[-1] == ")":
            continue
        results.append(line)
    return "\n".join(results)

def load_samples(filexml):
    # try:
    tree = Et.parse(filexml)
    root = tree.getroot()
    samples = []
    for i in range(0, len(root)):
        sample = {'result': []}
        for j, e in enumerate(root[i]):
            if e.tag == "t1":
                sample['result'] = format_first_line(e.text.strip())
            elif e.tag == "t2":
                question = e.text.strip()
                sample['content'] = question if len(question) > 0 else None
        sample.update(
            {'index': root[i].attrib['id'], 'label': root[i].attrib.get('label', "N")})
        # filter the noise samples
        if sample['content'] is not None:
            samples.append(sample)
        else:
            print("[Important warning] samples {} is ignored".format(sample))
    return samples

def load_test_data_samples(path_folder_base, test_id):
    data = []
    test = load_samples(f"{path_folder_base}/riteval_{test_id}.xml")
    for file_path in glob.glob(f"{path_folder_base}/riteval_{test_id}.xml"):
        data = data + load_samples(file_path)
    return data


def load_all_data_samples(path_folder_base):
    data = []
    for file_path in glob.glob("{}/*.xml".format(path_folder_base)):
        data = data + load_samples(file_path)
    return data

def check_false_labels(pred, false_labels):
	for label in false_labels:
		if label in pred:
			return True
	return False

from tqdm import tqdm

def format_output(text):
	CLEANR = re.compile('<.*?>') 
	cleantext = re.sub(CLEANR, '', text)
	return cleantext.strip().lower()

def readfile(filename):
    f = open(filename)
    data = json.load(f)
    return data

def load_jsonl(file):
    with open(file) as f:
        data = [json.loads(line) for line in f]
    return data

In [2]:
# def prompting(prompt="{{premise}}\nBased on the previous passage, {{hypothesis}}? ", data=None):
#     return prompt.replace("{{premise}}", data['result']).replace('{{hypothesis}}', data['content'])
test_file = "../data/COLIEE2024statute_data-English/test/riteval_R04_en.xml"
dev_file = "../data/COLIEE2024statute_data-English/test/riteval_R03_en.xml"

In [7]:
from sentence_transformers import SentenceTransformer, util
query_encoder = SentenceTransformer('facebook-dpr-question_encoder-single-nq-base')
passage_encoder = SentenceTransformer('facebook-dpr-ctx_encoder-single-nq-base')

def dpr(testfile, path="../data/COLIEE2024statute_data-English/fewshot", aug_data=None):
    files = get_all_files_from_path(path)
    corpus = []
    content = []
    labels = []
    for data in files:
        if testfile in data:
            continue
        data = load_samples(data)
        for item in data:
            # corpus.append(item["result"].replace("\n", " ").strip())
            corpus.append(item["result"].strip())
            content.append(item["content"].strip())
            labels.append(item["label"].strip())

    if aug_data is not None:
        files = get_all_files_from_path(aug_data)
        for file in files:
            data = load_jsonl(file)
            for item in data:
                # corpus.append(item["result"].replace("\n", " ").strip())
                corpus.append(item["result"].strip())
                content.append(item["content"].strip().replace("\n", " "))
                labels.append(item["label"].strip())
                
    retrival_passage_embeddings = passage_encoder.encode(corpus)
    content_passage_embeddings = passage_encoder.encode(content)       
    print(len(corpus))
    return corpus, content, labels, retrival_passage_embeddings, content_passage_embeddings


In [4]:
from transformers import AutoTokenizer, BloomForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "google/flan-t5-xxl"
cache_dir = "/home/congnguyen/drive/.cache"
# cache_dir = ".cache"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(
		model_name, device_map="auto", cache_dir=cache_dir, torch_dtype=torch.float16, load_in_8bit=True
	)

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:09<00:00,  1.90s/it]


In [5]:
from tqdm import tqdm
import random

def format_output(text):
	CLEANR = re.compile('<.*?>') 
	cleantext = re.sub(CLEANR, '', text)
	return cleantext.strip().lower()
    
def few_shot_prompting(indexes, corpus, content, labels, prompt_template):
    result = ""
    count = 0
    for i in indexes:
        if "true" in prompt_template.lower():
            answer = "True"
        else:
            answer = "Yes"
        if "N" == labels[i]:
            if "true" in prompt_template.lower(): answer = "False"
            else: answer = "No"
        prompt = prompt_template.replace("{{premise}}", corpus[i]).replace('{{hypothesis}}', content[i]).replace('{{answer}}', answer)
        result += prompt
        count += 1
    return result

def few_shot_prompting2(indexes, corpus, content, labels, prompt_template):
    result = ""
    count = 0
    for i in indexes:
        answer = "Probability is {{yes_score}}, {{no_score}}"
        if "N" == labels[i]:
            no_score = round(random.uniform(0.70, 0.99), 2)
            yes_score = round((1 - no_score), 2)
        else:
            yes_score = round(random.uniform(0.70, 0.99), 2)
            no_score = round((1 - yes_score), 2)
        answer = answer.replace("{{yes_score}}", str(yes_score)).replace("{{no_score}}", str(no_score))
        prompt = prompt_template.replace("{{premise}}", corpus[i]).replace('{{hypothesis}}', content[i]).replace('{{answer}}', answer)
        result += prompt + "\n"
        count += 1
    return result
    
def get_q_retrival(indexes, content):
    q_retrival = []
    for i in indexes:
        q_retrival.append(content[i])
    return q_retrival
    
def check_query(query, indexes, content):
    q_retrival = get_q_retrival(indexes, content)
    query_embedding = query_encoder.encode(query)
    passage_embeddings = passage_encoder.encode(q_retrival)
    scores = util.dot_score(query_embedding, passage_embeddings)
    indexes = torch.topk(scores, 3)
    print(indexes)
    
# prompt_template = "EXAM\n1. Answer by yes or no.\n\nDocument: {{premise}}\nQuestion: {{hypothesis}}?\nAnswer: {{answer}}\n\n"
# final_prompt = "EXAM\n1. Answer by yes or no.\n\nDocument: {{premise}}\nQuestion: {{hypothesis}}?\nAnswer: "
# zeroShot = "EXAM\n1. Answer by yes or no.\n\nDocument: {{premise}}\nQuestion: {{hypothesis}}? "

In [19]:
prompt_template = "EXAM\n1. Answer by yes or no.\n\nDocument: {{premise}}\nQuestion: {{hypothesis}}? \nAnswer: {{answer}}\n\n"
final_template = "EXAM\n1. Answer by yes or no.\n\nDocument: {{premise}}\nQuestion: {{hypothesis}}? "

path_file = "../data/COLIEE2024statute_data-English/test/"
aug_path = "../data/COLIEE2024statute_data-English/aug_sum/"

In [25]:
def predict(model, tokenizer, path_file, files=["riteval_R04_en"], output="../output/fewsho_detail/newpromt_"):
    for file in files:
        n_shot = 3
        test_file = path_file+file+".xml"
        f_result = open(output+file+"_acc.txt", "w", encoding="utf-8")
        f_prompt = open(output+file+"_prompt.txt", "w", encoding="utf-8")
        if "R05_en" in file:
            data = load_jsonl(test_file.replace(".xml", ".jsonl"))
        else:
            data = load_samples(test_file)
        corpus, content, labels, retrival_passage_embeddings, content_passage_embeddings = dpr(file, path_file, aug_path)
        count = 0
        for item in tqdm(data):
            id = item["index"]
            label = item["label"]
            hypothesis = item["content"]
            premise = item["result"]
            few_shot = ""
            #Important: You must use dot-product, not cosine_similarity
            # query_embedding = query_encoder.encode(premise)
            # scores = util.dot_score(query_embedding, retrival_passage_embeddings)
            # indexes = torch.topk(scores, 3).indices[0]
            # few_shot = few_shot_prompting(indexes, corpus, content, labels, prompt_template)
            text = few_shot + final_template.replace("{{premise}}", premise).replace("{{hypothesis}}", hypothesis)
            #############################################
            inputs = tokenizer(text, return_tensors="pt")["input_ids"].cuda()
            outputs = model.generate(inputs, max_new_tokens=10)
            output_text = format_output(tokenizer.decode(outputs[0]).replace(text, "").split("\n")[-1])
            if count < 1:
                print(text)
            if count < 20:
                print(output_text)
            if "yes" in output_text or "true" in output_text:
                output_text = "Y"
            else:
                output_text = "N"
            f_result.write(id+"\t"+output_text+"\t"+label+"\n")
            
            if output_text == label:
                f_prompt.write(id+": "+text+output_text+"\t"+label+"\n++++++++++++++++++++++++++++++\n")
                count+=1
            else:
                f_prompt.write(id+": "+text+output_text+"\t"+label+"\n------------------------------\n")
        print(file, count)


In [26]:
predict(model, tokenizer, path_file, ["riteval_R05_en", "riteval_R01_en","riteval_R02_en","riteval_R03_en","riteval_R04_en"], "../output/fewshot_detail/aug")

3440


  1%|█▍                                                                                                                                                         | 1/109 [00:00<00:36,  2.94it/s]

EXAM
1. Answer by yes or no.

Document: Article 537(1) If one of the parties promises in a contract to render a certain performance to a third party, the third party has the right to claim that performance directly from the obligor.
(2) The validity of the contract referred to in the preceding paragraph is not impaired even if a third party does not exist or a third party is not specified at the time of its formation.
(3) In the case referred to in paragraph (1), rights of the third party accrue when the third party has manifested intention of availing of the benefit of the contract under that paragraph to the obligor.
Question: The validity of a third party beneficiary contract made by A with C, the mother of B, to give land X owned by A to B gratuitously, is not impaired even if B is an unborn child as of the time of the formation of the contract.? 
yes


  2%|██▊                                                                                                                                                        | 2/109 [00:00<00:35,  3.04it/s]

yes


...

 26%|███████████████████████████████████████▌                                                                                                                  | 28/109 [00:09<00:27,  2.92it/s]

no


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 109/109 [00:40<00:00,  2.69it/s]

riteval_R05_en 84


ParseError: not well-formed (invalid token): line 1, column 0 (<string>)

In [14]:
prompt_template = "EXAM\n1. Answer by yes or no.\n\nDocument: {{premise}}\nQuestion: {{hypothesis}}? \nAnswer: {{answer}}\n\n"
final_template = "EXAM\n1. Answer by yes or no.\n\nDocument: {{premise}}\nQuestion: {{hypothesis}}? "
test_file = "../data/COLIEE2024statute_data-English/test/riteval_R04_en.xml"
corpus, content, labels, retrival_passage_embeddings, content_passage_embeddings = dpr("R04")

# final_prompt = "{{premise}}\nQuestion: {{hypothesis}}?\nAnswer "
# prompt_template
data = load_samples(test_file)
print(len(data))
prompt_template

1105
101


'EXAM\n1. Answer by yes or no.\n\nDocument: {{premise}}\nQuestion: {{hypothesis}}? \nAnswer: {{answer}}\n\n'

In [15]:
count = 0
for item in tqdm(data):
    label = item["label"].strip()
    retrival = item["result"].strip()
    query = item["content"].strip()
    text = ""
    id = item["index"]
    text = ""
    #Important: You must use dot-product, not cosine_similarity
    query_embedding = query_encoder.encode(retrival)
    scores = util.dot_score(query_embedding, retrival_passage_embeddings)
    indexes = torch.topk(scores, 3).indices[0]
    text = few_shot_prompting(indexes, corpus, content, labels, prompt_template)
    
    text += final_template.replace("{{premise}}", retrival).replace('{{hypothesis}}', query)
    inputs = tokenizer(text, return_tensors="pt")["input_ids"].cuda()
    outputs = model.generate(inputs, max_new_tokens=5)
    output_text = format_output(tokenizer.decode(outputs[0]).replace(text, "").split("\n")[-1])

    ##########################
    # output = []
    # query = query.split(".")
    # if len(query) > 1:
    #     for i in range(0, len(query)-1):
    #         text = final_template.replace("{{premise}}", retrival).replace('{{hypothesis}}', query[i])
    #         # text = text.replace("\n", " ")
    #         inputs = tokenizer(text, return_tensors="pt")["input_ids"].cuda()
    #         outputs = model.generate(inputs, max_new_tokens=5)
    #         output_text = format_output(tokenizer.decode(outputs[0]).replace(text, "").split("\n")[-1])
    #         print(text)
    #         output.append(output_text)
    # else:
    #     text += final_template.replace("{{premise}}", retrival).replace('{{hypothesis}}', query)
    #     # text = text.replace("\n", " ")
    #     inputs = tokenizer(text, return_tensors="pt")["input_ids"].cuda()
    #     outputs = model.generate(inputs, max_new_tokens=5)
    #     output_text = format_output(tokenizer.decode(outputs[0]).replace(text, "").split("\n")[-1])
    #     output.append(output_text)

    # output_text = output[0]
    # for k in output:
    #     if output_text != k:
    #         output_text = "false"
    # print(output, output_text, label)
    #############################
    
    ### Write
    if count < 1:
        print(text)
        
    if "yes" in output_text or "true" in output_text:
        output_text = "Y"
    else:
        output_text = "N"
    if output_text == label:
        count+=1
    # else:
    #     print(text)
    #     output = []
    #     query = query.split(".")
    #     if len(query) > 1:
    #         for i in range(0, len(query)-1):
    #             text = final_template.replace("{{premise}}", retrival).replace('{{hypothesis}}', query[i])
    #             # text = text.replace("\n", " ")
    #             inputs = tokenizer(text, return_tensors="pt")["input_ids"].cuda()
    #             outputs = model.generate(inputs, max_new_tokens=5)
    #             output_text = format_output(tokenizer.decode(outputs[0]).replace(text, "").split("\n")[-1])
    #             print(text)
    #             output.append(output_text)
    #     print(output)  
    #     print("==========================================")

  1%|█▌                                                                                                                                                         | 1/101 [00:02<04:30,  2.71s/it]

EXAM
1. Answer by yes or no.

Document: Article 6
(1) A minor who is permitted to conduct one or multiple types of business has the same capacity to act as an adult as far as that business is concerned.
(2) In a case as referred to in the preceding paragraph, if there are grounds that make the minor unable to sustain that business, the legal representative may revoke or limit the permission therefor in accordance with the provisions of Part IV (Relatives).
Question: Assuming a person is a minor, then in regards a permitted, specific business, he/she has the legal capacity to act.? 
Answer: Yes

EXAM
1. Answer by yes or no.

Document: Article 6
(1) A minor who is permitted to conduct one or multiple types of business has the same capacity to act as an adult as far as that business is concerned.
(2) In a case as referred to in the preceding paragraph, if there are grounds that make the minor unable to sustain that business, the legal representative may revoke or limit the permission ther

 32%|████████████████████████████████████████████████▊                                                                                                         | 32/101 [01:25<03:04,  2.68s/it]


KeyboardInterrupt: 

In [14]:
def write_test(data, output_file):
    f = open(output_file, "w", encoding="utf-8")
    for k in data:
        f.write(k.strip()+" "+data[k]+" "+"CAPTAIN\n")

In [1]:
from datasets import load_dataset
from random import randrange
import os

# for i in range(len(test_dataset["content"])):
output_folder = "../output/final/fewshot_cot/R02.task4.CAPTAIN3_riteval_R02_en.xml"
if not os.path.exists(output_folder):
	os.makedirs(output_folder)

files = ["riteval_H30_en", "riteval_R01_en", "riteval_R02_en", "riteval_R04_en", "riteval_R05_en"]
acc = {}
input_file = "../data/COLIEE2024statute_data-English/test/"
for file in files:
    count = 0
    test_file = input_file+file+".xml"
    data = load_samples(test_file)
    result = {}
    corpus, content, labels, retrival_passage_embeddings, content_passage_embeddings = dpr(file)
    for item in tqdm(data):
        id = item["index"]
        label = item["label"]
        retrival = item["result"]
        query = item["content"]
        #Important: You must use dot-product, not cosine_similarity
        query_embedding = query_encoder.encode(retrival)
        scores = util.dot_score(query_embedding, retrival_passage_embeddings)
        indexes = torch.topk(scores, 3).indices[0]
        text = few_shot_prompting(indexes, corpus, content, labels, prompt_template)
        
        text += final_template.replace("{{premise}}", retrival).replace('{{hypothesis}}', query)
        inputs = tokenizer(text, return_tensors="pt")["input_ids"].cuda()
        outputs = model.generate(inputs, max_new_tokens=5)
        output_text = format_output(tokenizer.decode(outputs[0]).replace(text, "").split("\n")[-1])
        print(output_text)
        output_text = format_output(output_text)
        answer = "N"
        if  "true" in output_text or "yes" in output_text:
            answer = "Y"
        if answer == label:
            count += 1
        result.update({id: answer})
    key = file.split("_")[1]
    outfile_path = f"{output_folder}/{key}.task4.CAPTAIN3_{file}.xml"
    write_test(result, outfile_path)
    acc.update({file: count/len(data)})
write_json(outfile_path.replace(".xml", ".json"),acc)
    

/home/congnguyen/drive/miniconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'load_samples' is not defined

In [25]:
acc

{'riteval_H30_en': 0.6428571428571429,
 'riteval_R01_en': 0.6396396396396397,
 'riteval_R02_en': 0.7160493827160493,
 'riteval_R04_en': 0.8415841584158416,
 'riteval_R05_en': 0.6972477064220184}

In [27]:
text = "Claim X, the claim of the payment of the sales price of A against B, has a special agreement prohibiting the assignment, but C acquired Claim X while knowing the existence of the special agreement prohibiting the assignment. C made a demand to B for the performance to C by specifying a reasonable period of time, but B did not perform the obligation within that period. In this case, B may not refuse to perform the obligation to C based on the special agreement prohibiting the assignment.?"
text.split(".")

['Claim X, the claim of the payment of the sales price of A against B, has a special agreement prohibiting the assignment, but C acquired Claim X while knowing the existence of the special agreement prohibiting the assignment',
 ' C made a demand to B for the performance to C by specifying a reasonable period of time, but B did not perform the obligation within that period',
 ' In this case, B may not refuse to perform the obligation to C based on the special agreement prohibiting the assignment',
 '?']

In [22]:
def predict(model, tokenizer, path_file, files=["riteval_R01_en","riteval_R02_en","riteval_R03_en","riteval_R04_en"], output="../output/fewshot_detail/newpromt_"):
    for file in files:
        test_file = path_file+file+".xml"
        data = load_samples(test_file)
        result = []
        count = 0
        n_shot = 3
        f = open(output+file+"_fewshot_full.txt", "w", encoding="utf-8")
        f_prompt = open(output+file+"_prompt_full.txt", "w", encoding="utf-8")
        corpus, content, labels, retrival_passage_embeddings, content_passage_embeddings, query_encoder = dpr(file)
        for item in tqdm(data):
            label = item["label"].replace("\n", " ").strip()
            retrival = item["result"].replace("\n", " ").strip()
            query = item["content"].replace("\n", " ").strip()
            text = ""
            id = item["index"]
            #Important: You must use dot-product, not cosine_similarity
            query_embedding = query_encoder.encode(retrival)
            scores = util.dot_score(query_embedding, retrival_passage_embeddings)
            indexes = torch.topk(scores, 3).indices[0]
            # text = few_shot_prompting(query, indexes, corpus, content, labels, prompt_template, n_shot)
            text = few_shot_prompting2(indexes, corpus, content, labels, prompt_template)
            # query_embedding = query_encoder.encode(query)
            # scores = util.dot_score(query_embedding, content_passage_embeddings)
            # indexes = torch.topk(scores, 1).indices[0]
            # text += few_shot_prompting(indexes, corpus, content, labels, prompt_template)
            
            text = text+final_prompt.replace("{{premise}}", retrival).replace('{{hypothesis}}', query)
            ##########################
            ## Zeroshot
            # text = zeroShot.replace("{{premise}}", item["result"]).replace('{{hypothesis}}', item["content"])
            ###########################
            # if count < 1:
            #     print("------------------------------")
            #     # print(prompt_template)
            #     print(text)
                # print(final_prompt)
            # print(text)
            inputs = tokenizer(text, return_tensors="pt")["input_ids"].cuda()
            outputs = model.generate(inputs, max_new_tokens=10)
            output_text = format_output(tokenizer.decode(outputs[0]).replace(text, "").split("\n")[-1])
            print(output_text)
            ### Write
            if "yes" in output_text or "true" in output_text:
                output_text = "Y"
            else:
                output_text = "N"
            if output_text == label:
                count+=1
            ###########
            f.write(id+"\t"+output_text+"\t"+label+"\n")
            f_prompt.write(id+"\t"+text+"=====================================\n")
        print(count/len(data))


# Evaluate R0-R4

In [73]:
path_file = "../data/COLIEE2024statute_data-English/fewshot/"

In [76]:
list_prompt = readfile("../data/prompt.json")
# list_prompt

In [77]:
def prompting(premise, hypothesis, template=None):
    text = template.replace("{{premise}}", premise).replace("{{hypothesis}}", hypothesis)
    return text

def writefile(data, filename):
    # Serializing json
    json_object = json.dumps(data, indent=1)
    # Writing to sample.json
    with open(filename, "w") as outfile:
        outfile.write(json_object)
        
def few_shot_prompting(indexes, corpus, content, labels, prompt_template):
    result = ""
    for i in indexes:
        if "true or false" in prompt_template.lower():
            answer = "True"
            if "N" == labels[i]:
                answer = "False"
        else:
            answer = "Yes"
            if "N" == labels[i]:
                answer = "No"
        prompt = prompt_template.replace("{{premise}}", corpus[i]).replace('{{hypothesis}}', content[i]).replace('{{answer}}', answer)
        result += prompt
    return result

In [ ]:
def predict(model, tokenizer, path_file, passage_embeddings, files=["riteval_R01_en","riteval_R02_en","riteval_R03_en","riteval_R04_en"], output="../output/accuracy2/newpromt_"):
    for file in files:
        test_file = path_file+file+".xml"
        f = open(output+file+".txt", "w", encoding="utf-8")
        data = load_samples(test_file)
        
        acc = {}
        for template_prompt in list_prompt:
            template_prompt = template_prompt["prompt"]
            query_prompt = template_prompt+"\nAnswer: "
            prompt_template = template_prompt+"\nAnswer: {{answer}}\n\n"
            
            result = []
            count = 0
            for item in tqdm(data):
                label = item["label"]
                hypothesis = item["content"]
                premise = item["result"]
                #Important: You must use dot-product, not cosine_similarity
                query_embedding = query_encoder.encode(premise)
                scores = util.dot_score(query_embedding, passage_embeddings)
                indexes = torch.topk(scores, 3).indices[0]
                few_shot = few_shot_prompting(indexes, corpus, content, labels, prompt_template)
                text = few_shot + prompting(premise, hypothesis, query_prompt)
                #############################################
                inputs = tokenizer(text, return_tensors="pt")["input_ids"].cuda()
                outputs = model.generate(inputs, max_new_tokens=10)
                output_text = format_output(tokenizer.decode(outputs[0]).replace(text, "").split("\n")[-1])
                f.write(text)
                f.write("\n3-shot answer:"+output_text+"\n")
                f.write("=========================================\n")
                if count < 1:
                    print(text)
                # if count < 3:
                #     print(output_text)
                if "yes" in output_text or "true" in output_text:
                    output_text = "Y"
                else:
                    output_text = "N"
                if output_text == label:
                    count+=1
                # print("predict label: ", output_text, "label: ", label)
            # print("=======================================")
            # print(template_prompt)
            # print(count, "/", len(data))
            acc.update({template_prompt: count/len(data)})
            print(acc)
        writefile(acc, output+file+".json")
        f.close()
predict(model, tokenizer, path_file, passage_embeddings, ["riteval_R01_en","riteval_R02_en","riteval_R03_en","riteval_R04_en"], "../output/fewshot_detail/")

  1%|██                                                                                                                                                                                                                                        | 1/111 [00:00<01:10,  1.55it/s]

Article 5
(1) A minor must obtain the consent of the minor's legal representative to perform a juridical act;provided, however, that this does not apply to a juridical act for merely acquiring a right or being released from an obligation.
(2) A juridical act in contravention of the provisions of the preceding paragraph is voidable.
(3) Notwithstanding the provisions of paragraph (1), a minor may freely dispose of property that the legal representative has permitted the minor to dispose of for a specified purpose, to an extent that falls within the scope of that purpose. The same applies if the minor disposes of property that the legal representative has permitted the minor to dispose of without specifying a purpose.
Question: A minor can freely dispose of assets the statutory agent permits the disposition without specifying the purpose. True or False?
Answer: True

Article 5
(1) A minor must obtain the consent of the minor's legal representative to perform a juridical act;provided, how

  2%|████▏                                                                                                                                                                                                                                     | 2/111 [00:01<01:06,  1.63it/s]

Article 9 A juridical act performed by an adult ward is voidable;provided, however, that this does not apply to the purchase of daily necessities or to any other act involved in day-to-day life.
Question: If an adult ward acquires land by gift, the guardian of the adult ward may not rescind the gift. True or False?
Answer: False

Article 9
A juridical act performed by an adult ward is voidable;provided, however, that this does not apply to the purchase of daily necessities or to any other act involved in day-to-day life.
Question: The purchase of daily household items may not be rescinded, even if an adult ward performed the act. True or False?
Answer: True

Article 9
A juridical act performed by an adult ward is voidable;provided, however, that this does not apply to the purchase of daily necessities or to any other act involved in day-to-day life.
Question: In cases where an adult ward receives the gift of a building, the adult ward cannot rescindthe relevant gift contract. True or F

  3%|██████▎                                                                                                                                                                                                                                   | 3/111 [00:03<02:38,  1.47s/it]

...

  4%|████████▍                                                                                                                                                                                                                                 | 4/111 [00:04<02:11,  1.23s/it]

Sentence 1: Article 15 (1) The family court may decide to commence an assistance in respect of a person whose capacity to appreciate their own situation is inadequate due to a mental disorder, at the request of the person in question, that person's spouse, that person's relative within the fourth degree of kinship, the guardian, the guardian's supervisor, the curator, the curator's supervisor, or a public prosecutor; provided, however, that this does not apply to a person with respect to whom there are grounds as prescribed in Article 7 or the main clause of Article 11.
(2) The issuance of a decision for commencement of assistance at the request of a person other than the person in question requires the consent of the person in question.

Sentence 2: The issuance of a decision for commencement of assistance at the request of a person other than the person in question requires the consent of the person in question unless the family court finds it to be suitable.

If the first sentence i

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 111/111 [01:17<00:00,  1.43it/s]


{'{{premise}}\nQuestion: {{hypothesis}} True or False?': 0.8468468468468469, '{{premise}}\nQuestion: {{hypothesis}} True or False? ': 0.8468468468468469, '{{premise}} Based on that information, is the claim: "{{hypothesis}}" true or false?': 0.8288288288288288, 'EXAM\n1. Answer by yes or no.\n\nDocument: {{premise}}\nQuestion: {{hypothesis}}? ': 0.8468468468468469, '{{premise}}\nBased on the previous passage, {{hypothesis}}? ': 0.8468468468468469, 'Premise: {{premise}}\n\nHypothesis: {{hypothesis}}\n\nDoes the premise entail the hypothesis? Yes or no ': 0.7567567567567568, 'Premise: {{premise}}\nHypothesis: {{hypothesis}}\nIs the hypothesis entailed by the premise? Yes or no ': 0.7657657657657657, 'Sentence 1: {{premise}}\n\nSentence 2: {{hypothesis}}\nIs this second sentence entailed by the first sentence? Yes or No ': 0.8018018018018018, 'Sentence 1: {{premise}}\n\nSentence 2: {{hypothesis}}\n\nIf the first sentence is true, then is the second sentence true? True or False ': 0.810810

  0%|                                                                                                                                                                                                                                                  | 0/111 [00:00<?, ?it/s]